## QJE Tesseract Extraction

In [ ]:
import fitz

# for OCR using PyTesseract
import cv2                              # pre-processing images
import pytesseract                      # extracting text from images
import numpy as np
import matplotlib.pyplot as plt         # displaying output images
from PIL import Image
import regex

In [ ]:
zoom_x = 3.0 # horizontal zoom
zoom_y = 3.0 # vertical zoom
mat = fitz.Matrix(zoom_x, zoom_y)

In [ ]:
path='D:\\docs\\Masters\\Data\\QJE_data'

In [ ]:
investigate=Merged[(Merged.year<=2010)&(Merged.year>=2000)]
for i in investigate.index:
    print(Merged.iloc[i]['stable_url'])
    if pd.isna(Merged.iloc[i]['affiliations'])==True:
        print(Merged.iloc[i]['Jstor_authors'])
        print(Merged.iloc[i]['Jstor_title'])
        print(Merged.iloc[i]['content_type'])

In [ ]:
for i in investigate.index:
    filepath=path+'\\'+investigate.iloc[i]['stable_url'].split('/')[-1]+'.pdf'
    if os.path.exists(filepath):
        doc=fitz.open(filepath)
        print(investigate.iloc[i]['year'])
        print(investigate.iloc[i]['issue'])
        print(investigate.iloc[i]['volume'])
        print(investigate.iloc[i]['Jstor_authors'])
        print(investigate.iloc[i]['Jstor_title'])
        for page in doc:
            #if (page.number == 1) or (page.number==(doc.page_count-1)):
            png = path+"\\" + investigate.iloc[i]['stable_url'].split('/')[-1].split('.')[0] + '_page-%i.png' % page.number
            print(png)
            #if os.path.exists(png)==True:
            pix = page.get_pixmap(matrix=mat)
            pix.save(png)
            #parsed[page.number]=[]

            original_image = cv2.imread(png)

            # convert the image to grayscale
            gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

            plt.figure(figsize=(25, 15))
            plt.imshow(gray_image, cmap='gray')
            plt.show()

In [ ]:
import string
def converter(teststring):
    teststring=teststring.replace('.','[.]')
    print(teststring)
    #for i in range(len(teststring)):
    #    if teststring[i] == '.':
    #        teststring=teststring[0:i-1]+'.'+teststring[i:]
    #teststring=teststring.replace('I','.')
            
    for i in teststring:
        if (i not in string.ascii_lowercase) & (i not in string.ascii_uppercase) & (i !=' ') & (i!='.') & (i!='[') & (i!=']'):
            teststring=teststring.replace(i,'.')
    return teststring


In [ ]:
def getNumberofPages(text):
    if pd.isna(text)==False:
        if re.search('\d',text):
            temp=text.split(',')
            pages=0
            print(temp)
            for m in temp:
                if '-' in m:
                    t=str(m).split('-')
                    pages=pages+int(re.sub('\D','',t[1]))-int(re.sub('\D','',t[0]))+1
                else:
                    pages+=1
            return pages
    return 1



In [ ]:
def generate_refs(SCANNED_FILE, mat, path, k_val, keyword):
    doc = fitz.open(SCANNED_FILE)
    parsed={}
    references={}
    found=0
    for page in reversed(doc):
        if (page.number >= 1):
            png = path+"\\" + SCANNED_FILE.split('\\')[-1].split('.')[0] + '_page-%i.png' % page.number
            if os.path.exists(png)==False:
                pix = page.get_pixmap(matrix=mat)
                print(png)
                pix.save(png)

            parsed[page.number]=[]
            references[page.number]=[]

            original_image = cv2.imread(png)
            # convert the image to grayscale
            gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

            #plt.figure(figsize=(25, 15))
            #plt.imshow(gray_image, cmap='gray')
            #plt.show()

            # Performing OTSU threshold
            ret, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

            #plt.figure(figsize=(25, 15))
            #plt.imshow(threshold_image, cmap='gray')
            #plt.show()

            rectangular_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (k_val, k_val))

            # Applying dilation on the threshold image
            dilated_image = cv2.dilate(threshold_image, rectangular_kernel, iterations = 1)

            #plt.figure(figsize=(25, 15))
            #plt.imshow(dilated_image)
            #plt.show()

            # Finding contours
            contours, hierarchy = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            # Creating a copy of the image
            copied_image = original_image.copy()

            mask = np.zeros(original_image.shape, np.uint8)
            #i=1
            # Looping through the identified contours
            # Then rectangular part is cropped and passed on to pytesseract
            # pytesseract extracts the text inside each contours
            # Extracted text is then written into a text file
            for cnt in contours:
                x, y, w, h = cv2.boundingRect(cnt)
                # Cropping the text block for giving input to OCR
                cropped = copied_image[y:y + h, x:x + w]
                # Apply OCR on the cropped image
                text = pytesseract.image_to_string(cropped, lang='lat', config='--oem 3 --psm 4')
                #print(i)
                print(text)
                parsed[page.number].append(text)
                print(regex.search(keyword,text.upper()))
                if regex.search(keyword, text.upper()) is not None:
                    print('found')
                    return {'found': parsed}
                #masked = cv2.drawContours(mask, [cnt], 0, (255, 255, 255), -1)
                #i=i+1
    return {'raw': parsed}

In [ ]:
def generate_pngs(SCANNED_FILE, mat, path, k_val, keyword, ignore):
    doc = fitz.open(SCANNED_FILE)
    parsed={}
    references={}
    found=0
    for page in reversed(doc):
        if (page.number >= 1):
            png = path+"\\" + SCANNED_FILE.split('\\')[-1].split('.')[0] + '_page-%i.png' % page.number
            if os.path.exists(png)==False:
                pix = page.get_pixmap(matrix=mat)
                print(png)
                pix.save(png)

            parsed[page.number]=[]
            references[page.number]=[]

            original_image = cv2.imread(png)
            # convert the image to grayscale
            gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

            #plt.figure(figsize=(25, 15))
            #plt.imshow(gray_image, cmap='gray')
            #plt.show()

            # Performing OTSU threshold
            ret, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

            #plt.figure(figsize=(25, 15))
            #plt.imshow(threshold_image, cmap='gray')
            #plt.show()

            rectangular_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (k_val, k_val))

            # Applying dilation on the threshold image
            dilated_image = cv2.dilate(threshold_image, rectangular_kernel, iterations = 1)

            #plt.figure(figsize=(25, 15))
            #plt.imshow(dilated_image)
            #plt.show()

            # Finding contours
            contours, hierarchy = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            # Creating a copy of the image
            copied_image = original_image.copy()

            mask = np.zeros(original_image.shape, np.uint8)
            #i=1
            # Looping through the identified contours
            # Then rectangular part is cropped and passed on to pytesseract
            # pytesseract extracts the text inside each contours
            # Extracted text is then written into a text file
            for cnt in contours:
                x, y, w, h = cv2.boundingRect(cnt)
                # Cropping the text block for giving input to OCR
                cropped = copied_image[y:y + h, x:x + w]
                # Apply OCR on the cropped image
                text = pytesseract.image_to_string(cropped, lang='lat', config='--oem 3 --psm 4')
                #print(i) 
                #print(regex.search(keyword,text.upper()))
                if text=='':
                    #print('empty af')
                    continue
                
                parsed[page.number].append(text)    
                
                if regex.search(ignore, text.upper()) is not None:
                    print('000110000')
                    #print(regex.search(ignore, text.upper()))
                    #print(text)
                    continue
                
                if regex.search('[a-zA-Z]', text.upper()) is None:
                    print('00020000')
                    #print(text)
                    continue
                    
                print(text)
                
                if found ==1:
                    print('found')
                    if regex.search('[a-zA-Z]', text.upper()) is not None:
                        
                        return {'found':text,'raw': parsed}
                if regex.search(keyword, text.upper()) is not None:
                    print('found')
                    found=1
                
                #masked = cv2.drawContours(mask, [cnt], 0, (255, 255, 255), -1)
                #i=i+1
    return {'raw': parsed}

In [ ]:

SCANNED_FILE=path+'\\40961018.pdf'
SCANNED_FILE=path+'\\25098840.pdf'
SCANNED_FILE=path+'\\27867508.pdf'
#SCANNED_FILE=path+'\\2586940.pdf'
#SCANNED_FILE=path+'\\2586895.pdf'
#SCANNED_FILE=path+'\\2696451.pdf'
SCANNED_FILE=path+'\\40961014.pdf'
SCANNED_FILE=path+'\\2951257.pdf'
t0=time.time()
ignore='((ARE GHETTOS GOOD OR BAD?){e<=3}(\n| \d)|^(THIS CONTENT DOWNLOADED){e<=3}|^(QUARTERLY JOURNAL OF ECONOMICS){e<=3}\n|^\d)'
affiliations=generate_pngs(SCANNED_FILE, mat, path, 30, '(^|\n)(REFERENCES){e<=3}(\n| )', ignore)
t1=time.time()
total=t1-t0
print(total)
affiliations

In [ ]:
dict={}

In [ ]:
t0=time.time()

for i in Merged[(Merged['year']>=1977) & (Merged['year']<=1979) & (Merged['content_type']!='MISC') & (Merged['content_type']!='Review')].index:
    if Merged.iloc[i]['Jstor_authors'] is not NaN: 
        if "Suggested by" not in Merged.iloc[i]['Jstor_authors']:
            authors=str(Merged.iloc[i]['Jstor_authors']).replace(' and ',', ').replace("  ",' ').split(',')
            filepath=path+'\\'+Merged.iloc[i]['stable_url'].split('/')[-1]+'.pdf'
            if os.path.exists(filepath)==True:
                print(filepath)
                print(Merged.iloc[i]['year'])
                print(authors)
                ignore='(('+Merged.iloc[i]['Jstor_title'].upper()+'){e<=3}(\n| \d| )|^(THIS CONTENT DOWNLOADED){e<=3}|^(QUARTERLY JOURNAL OF ECONOMICS){e<=3}\n|^\d)'
                affiliations=generate_pngs(filepath, mat, path, 38, '(^|\n)(REFERENCES){e<=3}(\n| )', ignore)                
                print(affiliations)
                dict[Merged.iloc[i]['stable_url'].split('/')[-1]]={'affiliations': affiliations, 
                                                                   'content_type': Merged.iloc[i]['content_type'], 
                                                                   'authors': authors, 
                                                                   'stable_url': Merged.iloc[i]['stable_url']}
            else:
                dict[Merged.iloc[i]['stable_url'].split('/')[-1]]='PDF not available, download at '+ Merged.iloc[i]['stable_url']
t1=time.time()
total=t1-t0
print(total)
print(i)

In [ ]:
import json
with open(path+'//QJE_aff_ref_output_1977_1989.json','w') as fp:
    json.dump(dict, fp)

In [ ]:
count=0
for l in dict.keys():
    if 'found' in dict[l]['affiliations'].keys():
        count+=1
    else:
        print(l)
print(len(dict))
print(count)